# Сверточные нейронные сети

Когда речь заходит об обработке изображений, то используется особая архитектура нейронных сетей - Сверточные нейронные сети

Каждый фильтр сканирует все пространство входного сигнала. CNN инвариантны к местоположению объекта.

Вычисленные признаки (n_channels) предполагается анализировать на более крупном масштабе:
Для этого размерность карт признаков сокращают с помощью одной из операций:

- MaxPooling - отбор наибольших значений;
- MinPooling - отбор наименьших значений;
- AveragePooling - отбор средних значений;


После MaxPooling карта признаков сокращается в 2 раза. Зачем это нужно? Для обобщения, цель выполнять анализ признаков на все более крупном масштабе. Получаем анализ данных на более крупном масштабе и благодаря этому нейроны следующего слоя способны выделять более общие признаки на изображении.

Идея: Двумерный входной сигнал изображения пропускается через фильтр. Фильтр имеет веса + bias. Далее накладываем фильтр на входной сигнал и формируем выходное значени:

$$z_{11} = \omega_{11} \cdot x_{11} + \omega_{12} \cdot x_{12} + ... + \omega_{33} \cdot x_{33} + \omega_0$$

$$u_{11} = \sigma(z_{11})$$

 - получаем 1-е выходное значение $u_{11}$ и далее смешаем окно и считаем следующее число. В результате получаем **карту признаков**. Процесс продолжается с другим ядром (фильтром). 

Классы сверточнных слоев:
- nn.Conv1d - для одномерной свертки
- nn.Conv2d - для двумерной свертки (например, изображение)
- nn.Conv3d - для трехмерной свертки (например, снимок МРТ)

-----------------------------------------------------------

- nn.MaxPool1d - для одномерного сигнала
- nn.MaxPool2d - для двумерного сигнала
- nn.MaxPool3d - для трехмерного сигнала

## Conv2d

На вход сверточного слоя, который определяется классом Conv2d, должен подаваться тензор в следующем формате:
(batch, channels, H, W)

- batch - размер пакета
- channels - число каналов в входном тензоре
- H - число строк
- W - число столбцов


Сам класс Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):

- если подается полноценное изображение в цветах RGB, nj in_channels = 3, если изображение в градациях серого, то in_channels = 1
- out_channels - число выходных каналов (число карты признаков), при этом каждая карта, которая является результатом применения фильтра к входному тензору в общем случае будет иметь размерность $H_m, W_m$, которые меньше, чем входные размеры

На выходе будем иметь размерность: (batch, out_channels, $H_m, W_m$)
- kernel_size - размер сверточного слоя, то есть все ядра в количестве out_channels будут иметь один размер;

А результирующий размер каждого ядра: (in_channels, kernel_size)

- stride - шаг сканирования фильтра (default: stride = 1)
- padding - расширение ядра (добавление нулей к входному тензору, default: padding=0)

Если padding = 'same', то размеры карт-признаков будут такими же, как и размеры входного тензора, то есть $H_m = H, W_m = W$

- bias к сумме добавляется смещение:

$$z_{n,m} = \omega_0 + \sum_{k=1}^{ch} \sum_{i=1}^{3} \sum_{j=1}^{3} \omega_{k,i,j} \cdot x_{k,i+n-1,j+m-2}$$

В ряде случаев, bias=False стоит убирать, когда используется batch_normalization


MaxPool2d(kernel_size, stried=None) часто применяют после сверточных слоев:

- kernel_size - размер области выделения
- stried - задает смещение областей

на выходе будем иметь размерность: (batch, channels, H // 2, W // 2)

In [8]:
import os
import json
from PIL import Image

import torch
import torch.utils.data as data
import torchvision.transforms.v2 as tfs
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class SunDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.path = os.path.join(path, "train" if train else "test")
        self.transform = transform

        with open(os.path.join(self.path, "format.json"), "r") as fp:
            self.format = json.load(fp)

        self.length = len(self.format)
        self.files = tuple(self.format.keys())
        self.targets = tuple(self.format.values())

    def __getitem__(self, item):
        path_file = os.path.join(self.path, self.files[item])
        img = Image.open(path_file).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(self.targets[item], dtype=torch.float32)
    
    def __len__(self):
        return self.length

In [15]:
transforms = tfs.Compose([tfs.ToImage(),
                          tfs.ToDtype(torch.float32, scale=True)])
d_train = SunDataset("dataset_reg", transform=transforms)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=True)

In [13]:
model = nn.Sequential(
    nn.Conv2d(3,32,3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2), # (batch, 32, 128, 128)
    nn.Conv2d(32,8,3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2), # (batch, 8, 64, 64)
    nn.Conv2d(8,4,3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2),  # (batch, 4, 32, 32)
    nn.Flatten(), # этот слой способен вытянуть все эти элементы (4, 32, 32) в единый вектор
    nn.Linear(4096, 128), # (batch, 4096 )
    nn.ReLU(),
    nn.Linear(128, 2)
)

In [14]:
optimizer = optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.001)
loss_function = nn.MSELoss()

In [17]:
epochs = 5
model.train()

for _e in range(epochs):
    loss_mean = 0
    lm_count = 0

    train_tqdm = tqdm(train_data, leave=True)
    for x_train, y_train in train_tqdm:
        predict = model(x_train)
        loss = loss_function(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1/lm_count * loss.item() + (1 - 1/lm_count) * loss_mean

        train_tqdm.set_description(f"Epoch {_e+1}/{epochs}, loss_mean={loss_mean:.3f}")

Epoch 5/5, loss_mean=9.595: 100%|██████████| 313/313 [03:32<00:00,  1.47it/s] 


In [19]:
st = model.state_dict()
torch.save(st,'model_sun.tar')

In [20]:
d_test = SunDataset("dataset_reg", train=False, transform=transforms)
test_data = data.DataLoader(d_test, batch_size=50, shuffle=False)

In [22]:
# тестирование НС
Q = 0
count = 0
model.eval()

test_tqdm = tqdm(test_data, leave=True)
for x_test, y_test in test_tqdm:
    with torch.no_grad():
        p = model(x_test)
        Q += loss_function(p, y_test).item()
        count += 1

Q /= count
print(Q)

100%|██████████| 20/20 [00:08<00:00,  2.38it/s]

7.398188281059265


## VGG-16, VGG-19

Visual Geometry Group

from t